In [3]:
import pandas as pd
import seaborn as sns

In [1]:
import os
import networkx as nx
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import numpy as np
from random import random, seed as rnd_seed
from sklearn.preprocessing import MinMaxScaler

ModuleNotFoundError: No module named 'networkx'

In [6]:
dpath = '/home/fransua/Projects/prostate_Claudia/data/LNCaP/'
dpath = '/home/fransua/Projects/prostate_Claudia/data/LNCaP/'
dpath = '/data01/alexandros/algorithms/bsc_mount/FIMO2PPI/Mega_loops_1inter_normalised/tables/filtered'

In [4]:
rpath = '/home/fransua/Projects/prostate_Claudia/data/LNCaP/graph_tables/'

# Load SNPs

In [8]:
enha_snps = pd.read_csv(os.path.join(dpath, 'binding_sites_snp_enha.txt'), 
                        sep='\t')
# remove variant label
enha_snps['motif_alt_id'] = enha_snps['motif_alt_id'].apply(
    lambda x: x.replace('(var.2)', '').replace('(var.3)', ''))
# split heterodimer complexes
enha_snps['motif_alt_id'] = enha_snps['motif_alt_id'].apply(lambda x: x.split('::'))
enha_snps = enha_snps.explode('motif_alt_id')
# split fused complexes
enha_snps['motif_alt_id'] = enha_snps['motif_alt_id'].apply(lambda x: x.split('-'))
enha_snps = enha_snps.explode('motif_alt_id')
enha_snps.drop_duplicates(inplace=True)

In [6]:
prom_snps = pd.read_csv(os.path.join(dpath, 'GWAS/binding_sites_snp_prom.txt'), 
                        sep='\t')
# remove variant label
prom_snps['motif_alt_id'] = prom_snps['motif_alt_id'].apply(
    lambda x: x.replace('(var.2)', '').replace('(var.3)', ''))
# split heterodimer complexes
prom_snps['motif_alt_id'] = prom_snps['motif_alt_id'].apply(lambda x: x.split('::'))
prom_snps = prom_snps.explode('motif_alt_id')
# split fused complexes
prom_snps['motif_alt_id'] = prom_snps['motif_alt_id'].apply(lambda x: x.split('-'))
prom_snps = prom_snps.explode('motif_alt_id')
prom_snps.drop_duplicates(inplace=True)

In [11]:
enha_coords = pd.read_csv(os.path.join(dpath, 'enhancer_table.txt'), 
                        sep='\t')

In [12]:
enha_coords['anchor'] = (enha_coords['CHR_enhancer'].astype(str) + '_' +
                         enha_coords['start.norm_Enhancer'].astype(str) + '_' +
                         enha_coords['end.norm_Enhancer'].astype(str))

In [16]:
enha_table = pd.merge(enha_coords, enha_snps, on='anchor', how='outer')

In [10]:
enha_coords['anchor'] = (enha_coords['CHR_Promoter'].astype(str) + '_' +
                         enha_coords['start.Promoter'].astype(str) + '_' +
                         enha_coords['end.Promoter'].astype(str))
prom_table = pd.merge(enha_coords, prom_snps, on='anchor', how='outer')

In [18]:
enha_table[enha_table['rsid'] == "rs9879866"]

,gene.name,CHR_enhancer,start.Enhancer,end.Enhancer,start.norm_Enhancer,end.norm_Enhancer,CHR_Promoter,start.Promoter,end.Promoter,cluster,CTCF_cluster,GWAS_cluster,CTCF_binding,GWAS_SNP,anchor,motif_alt_id,rsid,type
19502,GATA2-AS1,3,127925000,127940000,127936000,127937000,3,128207545,128208544,8,+,+,Yes,Yes,3_127936000_127937000,VEZF1,rs9879866,ENHA
19504,GATA2-AS1,3,127925000,127940000,127936000,127937000,3,128207545,128208544,8,+,+,Yes,Yes,3_127936000_127937000,ZNF740,rs9879866,ENHA
19508,GATA2-AS1,3,127930000,127945000,127936000,127937000,3,128207545,128208544,8,+,+,Yes,Yes,3_127936000_127937000,VEZF1,rs9879866,ENHA
19510,GATA2-AS1,3,127930000,127945000,127936000,127937000,3,128207545,128208544,8,+,+,Yes,Yes,3_127936000_127937000,ZNF740,rs9879866,ENHA
19514,GATA2-AS1,3,127935000,127950000,127936000,127937000,3,128207545,128208544,8,+,+,Yes,Yes,3_127936000_127937000,VEZF1,rs9879866,ENHA
19516,GATA2-AS1,3,127935000,127950000,127936000,127937000,3,128207545,128208544,8,+,+,Yes,Yes,3_127936000_127937000,ZNF740,rs9879866,ENHA


In [19]:
list(enha_table[(enha_table['gene.name']=="GATA2-AS1") & 
                                   (enha_table['motif_alt_id']==node[5:])]['rsid'])[0]

NameError: name 'node' is not defined

## Load expression

In [11]:
expression = pd.read_csv(os.path.join(dpath, 'expression', 'DESeq_LNCaP_LHSAR.txt'), sep=' ')

## create more columns to describe edges

In [12]:
def label_edges(data):
    data['enhancer_1'] = data['edge'].apply(lambda x: x.split(';')[0]
                                            if 'ENHA_' in x.split(';')[0] else False)
    data['promoter_1'] = data['edge'].apply(lambda x: x.split(';')[0]
                                            if 'PROM_' in x.split(';')[0] else False)
    data['intermed_1'] = data['edge'].apply(lambda x: x.split(';')[0] 
                                            if 'ENHA_' not in x.split(';')[0] and 
                                               'PROM_' not in x.split(';')[0] else False)
    data['enhancer_2'] = data['edge'].apply(lambda x: x.split(';')[1]
                                            if 'ENHA_' in x.split(';')[1] else False)
    data['promoter_2'] = data['edge'].apply(lambda x: x.split(';')[1]
                                            if 'PROM_' in x.split(';')[1] else False)
    data['intermed_2'] = data['edge'].apply(lambda x: x.split(';')[1] 
                                            if 'ENHA_' not in x.split(';')[1] and 
                                               'PROM_' not in x.split(';')[1] else False)
    data['node_1'] = data['edge'].apply(lambda x: x.split(';')[0])
    data['node_2'] = data['edge'].apply(lambda x: x.split(';')[1])

    data['enhancer'] = data['edge'].apply(lambda x: 
                                          'ENHA_' in x.split(';')[0] or
                                          'ENHA_' in x.split(';')[1])
    data['promoter'] = data['edge'].apply(lambda x: 
                                          'PROM_' in x.split(';')[0] or
                                          'PROM_' in x.split(';')[1])
    data['intermed'] = data['edge'].apply(lambda x:
                                          ('ENHA_' not in x.split(';')[0] or 'ENHA_' not in x.split(';')[1]) 
                                          and 
                                          ('PROM_' not in x.split(';')[0] or 'PROM_' not in x.split(';')[1]))

In [13]:
for fnam in os.listdir(os.path.join(dpath, 'use_case_tables')):
    if not fnam.endswith('.txt'):
        continue
    name = fnam.split('.')[0]
    edges = pd.read_csv(os.path.join(dpath, 'use_case_tables', fnam), sep='\t')
    # remove edges prom-prom or enha-enha
    edges = edges[~((edges['edge'].apply(lambda x: x.count('ENHA_')) > 1) | 
                    (edges['edge'].apply(lambda x: x.count('PROM_')) > 1))]
    # add some labelling
    label_edges(edges)
    
    edges['druggable'] = (edges['drugs.targeting.both.proteins']!= '-') & (edges['drug.targets'] >= 1)

    enhancers = set(v for v in 
                    (list(edges[edges['enhancer']]['enhancer_1']) +  
                     list(edges[edges['enhancer']]['enhancer_2'])) 
                    if v)
    promoters = set(v for v in 
                    (list(edges[edges['promoter']]['promoter_1']) +
                     list(edges[edges['promoter']]['promoter_2']))
                    if v)
    intermeds = set(v for v in 
                    (list(edges[edges['intermed']]['intermed_1']) +  
                     list(edges[edges['intermed']]['intermed_2']))
                    if v)

    size_p = len(promoters)
    size_e = len(enhancers)

    # compute graph coords
    rnd_seed(1)        # or any integer
    np.random.seed(1)

    # build graph
    G = nx.Graph()
    for k in edges['edge'].apply(lambda x: x.split(';')):
        G.add_edge(*k)
    # define node coordinates of intermediates
    pos = nx.fruchterman_reingold_layout(G, weight=0.01, k=1, iterations=50)

    # X limits
    xprom = -1.1
    xenha = 1.1

    # add node cordinates for promoters
    x, y = [xprom - (p % 3) / 30 for p in range(size_p)], np.linspace(-1.1, 1.1, size_p)
    pos.update(dict((n, (i, j)) for i, j, n in zip(x, y, sorted(promoters))))
    # add node cordinates for enhancers
    x, y = [xenha + (p % 3) / 30 for p in range(size_e)], np.linspace(-1.1, 1.1, size_e)
    pos.update(dict((n, (i, j)) for i, j, n in zip(x, y, sorted(enhancers))))

    # put X-Y coordinates in edges
    edges['XYs-edge'] = list(zip(edges['node1'].apply(lambda x:pos[x]), 
                                 edges['node2'].apply(lambda x:pos[x])))
    
    tmp = []
    Gstat_headers = ['Betweenness_global', 'Degree_global',
                     'Betweenness_in_cluster', 'Betweenness_out_cluster',
                     'Betweenness_enrichment', 'Degree_in_cluster', 'Degree_out_cluster',
                     'Degree_enrichment', 'Betweenness_enrichment_pval',
                     'Degree_enrichment_pval', 'fisher_test_OR', 'fisher_test_pval']
    columns = ['protein', 'is_enhancer', 'is_promoter', 'is_intermediate', 
               'X-graph-coord', 'Y-graph-coord', 'degree',
               'expression_log2FoldChange', 'expression_padj', 'SNP-binding'] + Gstat_headers
    Gstats = [float('nan')] * len(Gstat_headers)
    edges_snp = np.array([False] * len(edges))
    for node in enhancers:
        try:
            snp = list(enha_table[(enha_table['gene.name']==name) & 
                                   (enha_table['motif_alt_id']==node[5:])]['rsid'])[0]
        except IndexError:
            snp = False
        count = len(edges[edges['edge'].apply(lambda x: node in x.split(';'))])
        OR = edges[edges['edge'].apply(lambda x: node in x.split(';'))]['fisher_test_edge_OR'].median()
        pv = edges[edges['edge'].apply(lambda x: node in x.split(';'))]['fisher_test_edge_pvalue'].median()
        Gstats[Gstat_headers.index('fisher_test_OR')] = OR
        Gstats[Gstat_headers.index('fisher_test_pval')] = pv
        # create data-frame
        s = expression['Row.names']==node[5:]
        tmp.append([node,                                                     # node name
                    True,                                                     # is enhancer
                    False,                                                    # is promoter
                    False,                                                    # is intermediate
                    pos[node][0],                                             # X coordinate
                    pos[node][1],                                             # Y coordinate
                    count,                                                    # number edges through
                    expression['log2FoldChange'].get(                         # expression fold change
                        expression[s].index[0] if any(s) else None, float('nan')),
                    expression['padj'].get(                                   # expression p-value change
                        expression[s].index[0] if any(s) else None, float('nan')),
                    snp,                                                     # SNP in protein binding site
                   ] + Gstats)

        # label edges connected to enhancer-SNPs
        if snp:
            for nprom in promoters:
                for ns_ in nx.all_simple_paths(G, node, nprom, cutoff=2):
                    try:
                        n1, n2, n3 = ns_
                    except ValueError:
                        n1, n2 = ns_
                        n3 = ''
                    # we only want edges that are connected to intermediate nodes (no enha-enha or prom-prom)
                    if n2.startswith('ENHA_'):
                        continue
                    edges_snp = edges_snp | (edges['node1'] == n1) & (edges['node2'] == n2)
                    edges_snp = edges_snp | (edges['node1'] == n2) & (edges['node2'] == n1)
                    if not n2.startswith('PROM_') or n3 == '':
                        edges_snp = edges_snp | (edges['node1'] == n3) & (edges['node2'] == n2)
                        edges_snp = edges_snp | (edges['node1'] == n2) & (edges['node2'] == n3)
    edges['SNP-path'] = edges_snp

    for node in promoters:
        try:
            snp = list(prom_table[(prom_table['gene.name']==name) & 
                                   (prom_table['motif_alt_id']==node[5:])]['rsid'])[0]
            print(name, psnp)
        except IndexError:
            snp = False
        count = len(edges[edges['edge'].apply(lambda x: node in x.split(';'))])
        OR = edges[edges['edge'].apply(lambda x: node in x.split(';'))]['fisher_test_edge_OR'].median()
        pv = edges[edges['edge'].apply(lambda x: node in x.split(';'))]['fisher_test_edge_pvalue'].median()
        Gstats[Gstat_headers.index('fisher_test_OR')] = OR
        Gstats[Gstat_headers.index('fisher_test_pval')] = pv
        # create data-frame
        s = expression['Row.names']==node[5:]
        tmp.append([node,                                                     # node name
                    False,                                                    # is enhancer
                    True,                                                     # is promoter
                    False,                                                    # is intermediate
                    pos[node][0],                                             # X coordinate
                    pos[node][1],                                             # Y coordinate
                    count,                                                    # number edges through
                    expression['log2FoldChange'].get(                         # expression fold change
                        expression[s].index[0] if any(s) else None, float('nan')),
                    expression['padj'].get(                                   # expression p-value change
                        expression[s].index[0] if any(s) else None, float('nan')),
                    snp,                                                     # SNP in protein binding site
                   ] + Gstats)

    for node in intermeds:        
        Gstats = list(edges[edges['intermediate_node']==node][Gstat_headers].to_numpy()[0])
        count = len(edges[edges['edge'].apply(lambda x: node in x.split(';'))])
        # create data-frame
        s = expression['Row.names']==node
        tmp.append([node,                                                     # node name
                    False,                                                    # is enhancer
                    False,                                                    # is promoter
                    True,                                                     # is intermediate
                    pos[node][0],                                             # X coordinate
                    pos[node][1],                                             # Y coordinate
                    count,                                                    # number edges through
                    expression['log2FoldChange'].get(                         # expression fold change
                        expression[s].index[0] if any(s) else None, float('nan')),
                    expression['padj'].get(                                   # expression p-value change
                        expression[s].index[0] if any(s) else None, float('nan')),
                    False,                                                     # SNP in protein binding site
                   ] + Gstats)
        

    # stores edges    
    edges = edges[[
        'edge', 'fisher_test_edge_OR', 'fisher_test_edge_pvalue',
        'SNP-path', 'XYs-edge', 'druggable']]

    # store nodes
    nodes = pd.DataFrame(tmp, columns=columns)
    nodes.to_csv(os.path.join(rpath, f'{name}_nodes.tsv'), sep='\t', index=False)
    edges.to_csv(os.path.join(rpath, f'{name}_edges.tsv'), sep='\t', index=False)